# 3. Point cloud segmentation with Deep Learning

As we mentioned in the previous section, we are going to classiffy all the points in forest scenes into 2 classes: **trees** and **DTM**. In order to do this, we are going to use a top DL model called [PointNet++](https://github.com/charlesq34/pointnet2).

A modified architecture of the model plus all the explained codes are available [within the contents of this workshop](https://github.com/LinoComesana/PCD_DL_WORKSHOP/tree/main/Deep_Learning/pointnet2).

## 3.1. Generation of training datasets

Taking the codes of the previous section as a basis, we can create a script which generates randomly and iteratively different forest point clouds. Here is a quick example that generates a dataset of 10 synthetic point clouds divided into 3 groups: train, test and prediction:

In [1]:
import numpy as np
import open3d as o3d
import os
import time

instante_inicial = time.time()

ruta_proyecto_workshop = os.getcwd()

print(os.listdir())


Number_of_pointclouds = 10

for XD in range(Number_of_pointclouds):

    eje_x = np.arange(0,100,0.1)
    eje_y = np.arange(0,100,0.1)

    SUPERFICIE = {}
    mesh_x, mesh_y = np.meshgrid(eje_x,eje_y)

    xyz = np.zeros((np.size(mesh_x), 3))
    xyz[:, 0] = np.reshape(mesh_x, -1)
    xyz[:, 1] = np.reshape(mesh_y, -1)


    nube_plano = o3d.geometry.PointCloud()
    nube_plano.points = o3d.utility.Vector3dVector(xyz)



    N_picos = np.random.randint(10,50)
    for q in range(N_picos):
        indices_puntos = np.arange(0,len(xyz),1)
        indice_punto_central_pico = np.random.choice(indices_puntos,size=1)
        punto_central = xyz[indice_punto_central_pico]
        nube_punto_central = o3d.geometry.PointCloud()
        nube_punto_central.points = o3d.utility.Vector3dVector(np.vstack((punto_central,
                                                                          punto_central)))
        buffer = np.random.randint(10,20)
        # Cojo todos los punntos dentro de ese buffer respecto al punto central:
        distancias = np.array(nube_plano.compute_point_cloud_distance(nube_punto_central))
        indices_dentro_buffer = np.where(distancias <= buffer)[0]

        # indices_cambios_altura = indices_puntos
        xyz[indices_dentro_buffer,2] = np.random.uniform(buffer/2.,buffer,size=len(indices_dentro_buffer)).T


    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)







    # We define some useful functions:
    import itertools

    Numero_puntos_superficie = 1e6

    def exponential_cov(x, y, params):
        return params[0] * np.exp( -0.5 * params[1] * np.subtract.outer(x, y)**2)

    def conditional(x_new, x, y, params):
        B = exponential_cov(x_new, x, params)
        C = exponential_cov(x, x, params)
        A = exponential_cov(x_new, x_new, params)
        mu = np.linalg.inv(C).dot(B.T).T.dot(y)
        sigma = A - B.dot(np.linalg.inv(C).dot(B.T))
        return(mu.squeeze(), sigma.squeeze())

    ordr = 4  # Orden del polinomio al que quiero ajustar en cada "frame"

    def matriz_minimos_cuadrados(x, y, order=ordr):
        """ generate Matrix use with lstsq """
        # Genero una matriz con los valores obtenidos por mínimos cuadrados
        ncolumnas = (order + 1)**2
        G = np.zeros((x.size, ncolumnas))
        ij = itertools.product(range(order+1), range(order+1))
        for k, (i, j) in enumerate(ij):
            G[:, k] = x**i * y**j
        return G


    puntos = np.copy(xyz)

    x, y, z = puntos.T # Hago la traspuesta
    x, y = x - x[0], y - y[0]  # Para mejorar la eficacia

    # Creamos la matriz que contiene las regresiones por punto:
    G = matriz_minimos_cuadrados(x, y, ordr)
    # Solve for np.dot(G, m) = z:
    # Quiero saber qué valores de m hacen que G·m = z (es decir, np.dot(G,m)=z)
    m = np.linalg.lstsq(G, z)[0]


    # Evaluamos en una nueva grid el ajuste que acabamos de hacer...
    nx, ny = int(np.sqrt(Numero_puntos_superficie)), int(np.sqrt(Numero_puntos_superficie))
    xx, yy = np.meshgrid(np.linspace(x.min(), x.max(), nx),
                          np.linspace(y.min(), y.max(), ny))


    GG = matriz_minimos_cuadrados(xx.ravel(), yy.ravel(), ordr)
    zz = np.reshape(np.dot(GG, m), xx.shape)

    # Convierto la superficie del fit a una nube de puntos:
    superficie_reconstruida = np.zeros((np.size(xx), 3))
    superficie_reconstruida[:, 0] = np.reshape(xx, -1)
    superficie_reconstruida[:, 1] = np.reshape(yy, -1)
    superficie_reconstruida[:, 2] = np.reshape(zz, -1)

    pcd2 = o3d.geometry.PointCloud()
    pcd2.points = o3d.utility.Vector3dVector(superficie_reconstruida)





    # Ahora le añadimos algo de ruido gaussiano a cada punto para hacerlo más real:
    ruido_x = np.random.normal(0,eje_x[2]-eje_x[1],len(superficie_reconstruida))
    ruido_y = np.random.normal(0,eje_y[2]-eje_y[1],len(superficie_reconstruida))
    # ruido_z = np.random.normal(0,0.2,len(superficie_reconstruida))

    superficie_reconstruida[:, 0] = np.reshape(superficie_reconstruida.take(0,1) + ruido_x, -1)
    superficie_reconstruida[:, 1] = np.reshape(superficie_reconstruida.take(1,1) + ruido_y, -1)
    # superficie_reconstruida[:, 2] = np.reshape(superficie_reconstruida.take(2,1) + ruido_z, -1)


    pcd_DTM = o3d.geometry.PointCloud()
    pcd_DTM.points = o3d.utility.Vector3dVector(superficie_reconstruida)

    lista_colores_suelo = [np.array([51/255,51/255,0/255]),
                     np.array([102/255,102/255,0/255]),
                     np.array([153/255,153/255,0/255]),
                     np.array([204/255,204/255,0/255]),
                     np.array([255/255,255/255,0/255]),
                     np.array([102/255,51/255,0/255]),
                     np.array([153/255,76/255,0/255]),
                     np.array([204/255,102/255,0/255])]

    colores_suelo = np.zeros((len(superficie_reconstruida),3))
    for e in range(len(colores_suelo)):
        colores_suelo[e] = lista_colores_suelo[np.random.choice(len(lista_colores_suelo))]

    pcd_DTM.colors = o3d.utility.Vector3dVector(colores_suelo)




    from LECTURAS_segmentacion_cilindrica import lectura_segmentaciones
    import os
    import math

    print(os.listdir())

    os.chdir(ruta_proyecto_workshop+'/data/trees_pcd')

    ruta_actual = os.getcwd()
    print(ruta_actual)

    SEGMENTOS = lectura_segmentaciones(ruta_actual)
    os.chdir(ruta_actual)

    numero_de_transformaciones_por_arbol = 3

    def rotar_punto(origen, punto, angulo):

        # Ángulo en radianes

        ox, oy = origen
        px, py = punto[0],punto[1]

        qx = ox + math.cos(angulo) * (px - ox) - math.sin(angulo) * (py - oy)
        qy = oy + math.sin(angulo) * (px - ox) + math.cos(angulo) * (py - oy)
        return np.array([qx, qy,punto[2]])

    SEGMENTOS_ARTIFICIALES = {}


    contador_arboles = 0
    for j in range(numero_de_transformaciones_por_arbol):
        for i in range(len(SEGMENTOS)):

            segmento = SEGMENTOS[i]

            # Todos los árboles que vayamos a crear se verán sometidos a, como mínimo, una
            # rotación en el plano XY. Para ello calculamos primero su eje de simetría:


            cgx = (1/len(segmento.points))*(np.sum(np.array(segmento.points).take(0,1)))
            cgy = (1/len(segmento.points))*(np.sum(np.array(segmento.points).take(1,1)))
            #cgz = (1/len(segmento.points))*(np.sum(np.array(segmento.points).take(2,1)))

            centro_de_masas = [cgx,cgy]

            puntos_rotados = []

            angulo = np.random.random() # En radianes lo cogemos

            for punto in segmento.points:
                puntos_rotados.append(rotar_punto(centro_de_masas, punto, angulo))

            puntos_rotados = np.array(puntos_rotados)

            segmento_rotado = o3d.geometry.PointCloud()
            segmento_rotado.points = o3d.utility.Vector3dVector(puntos_rotados)


            # Ahora le vamos a meter algo de ruido (pero sólo a los puntos que
            # estén por encima de una determinada altura).

            dispersion = np.mean(segmento_rotado.compute_point_cloud_distance(segmento))


            altura_media = np.mean(puntos_rotados[:,2])

            indices = np.where(puntos_rotados[:,2]>altura_media)[0]

            ruido_x = np.random.normal(0,dispersion,len(indices))
            ruido_y = np.random.normal(0,dispersion,len(indices))
            ruido_z = np.random.normal(0,0.5,len(indices))

            # print(indices)
            # print()
            # print(puntos_rotados)

            puntos_rotados[indices, 0] = np.reshape(puntos_rotados[indices, 0] + ruido_x, -1)
            puntos_rotados[indices, 1] = np.reshape(puntos_rotados[indices, 1] + ruido_y, -1)
            puntos_rotados[indices, 2] = np.reshape(puntos_rotados[indices, 2] + ruido_z, -1)


            # Volvemos a definir el nuevo árbol:
            arbol_artificial = o3d.geometry.PointCloud()
            arbol_artificial.points = o3d.utility.Vector3dVector(puntos_rotados)

            # Lo pintamos con un color:
            lista_colores_arbol = [np.array([0/255,51/255,0/255]),
                             np.array([0/255,102/255,0/255]),
                             np.array([0/255,153/255,0/255]),
                             np.array([0/255,204/255,0/255]),
                             np.array([0/255,255/255,0/255]),
                             np.array([51/255,255/255,51/255]),
                             np.array([102/255,255/255,102/255])]

            colores_arbol = np.zeros((len(puntos_rotados),3))
            for e in range(len(colores_arbol)):
                colores_arbol[e] = lista_colores_arbol[np.random.choice(len(lista_colores_arbol))]

            arbol_artificial.colors = o3d.utility.Vector3dVector(colores_arbol)




            # El False se lo pongo para designar que no ha sido seleccionado to-
            # davía:
            SEGMENTOS_ARTIFICIALES[contador_arboles] = [arbol_artificial,False]
            contador_arboles += 1



    import copy

    ARBOLES = []
    NUBES_ARTIFICIALES = {}
    #superficie = SUPERFICIES[0]
    superficie = pcd_DTM
    ARBOLES.append([])

    Numero_arboles_por_nube = 100

    puntos_superficie = np.array(superficie.points)


    SEGMENTOS_ARTIFICIALES_aux = copy.deepcopy(SEGMENTOS_ARTIFICIALES)





    contador = 0
    cuntudur = 0
    for j in range(Numero_arboles_por_nube):

        # Si ya hemos terminado de poner todos los árboles que teníamos
        # creados volvemos a empezar:
        if cuntudur == len(SEGMENTOS_ARTIFICIALES_aux):
            # print(SEGMENTOS_ARTIFICIALES[0])
            # SEGMENTOS_ARTIFICIALES_aux.clear()
            SEGMENTOS_ARTIFICIALES_aux = copy.deepcopy(SEGMENTOS_ARTIFICIALES)
            # print(SEGMENTOS_ARTIFICIALES[0])
            cuntudur = 0

        posible_ubicacion = puntos_superficie[np.random.choice(len(puntos_superficie))]

        indice = np.random.choice(len(SEGMENTOS_ARTIFICIALES_aux))
        # Cojo un árbol al azar:
        arbol = SEGMENTOS_ARTIFICIALES_aux[indice]
        # arbol = SEGMENTOS_ARTIFICIALES[10] # Ejemplo

        while arbol[1] == True:
            indice = np.random.choice(len(SEGMENTOS_ARTIFICIALES_aux))
            arbol = SEGMENTOS_ARTIFICIALES_aux[indice]
        if arbol[1] == False:
            cuntudur += 1

        arbol = arbol[0]
        # Le ponemosla etiqueta True porque ya ha sido seleccionado:
        SEGMENTOS_ARTIFICIALES_aux[indice][1] = True
        SEGMENTOS_ARTIFICIALES[indice][1] = False
        # if cuntudur == 1:
        #     print(SEGMENTOS_ARTIFICIALES_aux[indice][1])
        #     print(SEGMENTOS_ARTIFICIALES[indice][1])
        arbol.translate((posible_ubicacion),relative=False)

        puntos_arbol = np.array(arbol.points)
        minima_altura = puntos_arbol.take(2,1).min()

        color_arbol = np.array(arbol.colors)

        desfase_vertical = posible_ubicacion-minima_altura

        puntos_arbol[:, 2] = puntos_arbol.take(2,1) + desfase_vertical[2]
        arbol = o3d.geometry.PointCloud()
        arbol.points = o3d.utility.Vector3dVector(puntos_arbol)
        arbol.colors = o3d.utility.Vector3dVector(color_arbol)


        ARBOLES[-1].append(arbol)

        if contador == 0:

            nube_artificial = arbol+superficie
            contador += 1

        else:
            nube_artificial += arbol


    # Metemos todos los árboles que coleccionamos en un sólo elemento:
    arbol = o3d.geometry.PointCloud()

    for arbol_i in ARBOLES[-1]:
        arbol += arbol_i

    # Cambiamos tooodos los elementos por uno sólo que serán todos los ár-
    # boles:
    ARBOLES = arbol
    NUBES_ARTIFICIALES = nube_artificial





    os.chdir(ruta_proyecto_workshop+'/data/synthetic_point_clouds')
    try:
        os.mkdir('SYNTHETIC_DATASET')
    except FileExistsError:
        print('The folder already exists, so we pass')
    os.chdir('SYNTHETIC_DATASET')

    path_dataset = os.getcwd()
    
    try:
        os.mkdir('Nube_artificial_%s'%str(XD))
    except FileExistsError:
        pass
    os.chdir('Nube_artificial_%s'%str(XD))


    os.mkdir('numpy_arrays')
    os.chdir('numpy_arrays')
    
    # Too many creations of subfolders, I know but this is a modified version of a original simulator that I developed... Sorry ;)

    puntos_arboles = np.array(ARBOLES.points)
    with open("arboles.npy", 'wb') as f:    
        np.save(f, puntos_arboles)
    with open("DTM.npy", 'wb') as f:    
        np.save(f, puntos_superficie) 
    
    puntos_nube = np.vstack((puntos_arboles,puntos_superficie))
    with open("Nube_artificial_%s.npy"%str(XD), 'wb') as f:    
        np.save(f, puntos_superficie) 

        
    import shutil
    os.chdir(path_dataset)
    if XD <= 6:
        try:
            os.mkdir('TRAIN')
        except FileExistsError:
            pass
        original = os.getcwd()+'/Nube_artificial_%s'%str(XD)
        target = os.getcwd()+'/TRAIN/Nube_artificial_%s'%str(XD)
        shutil.move(original, target)
    elif 6 < XD < 9:
        try:
            os.mkdir('TEST')
        except FileExistsError:
            pass
        original = os.getcwd()+'/Nube_artificial_%s'%str(XD)
        target = os.getcwd()+'/TEST/Nube_artificial_%s'%str(XD)
        shutil.move(original, target)        
    else:
        try:
            os.mkdir('PREDICT')
        except FileExistsError:
            pass
        original = os.getcwd()+'/Nube_artificial_%s'%str(XD)
        target = os.getcwd()+'/PREDICT/Nube_artificial_%s'%str(XD)
        shutil.move(original, target)       
    
    instante_fin_creacion_nube = time.time()
    
    print('Nube generada. Transcurrido por ahora: ',(instante_fin_creacion_nube-instante_inicial)/60.,' min')
        
    os.chdir(ruta_proyecto_workshop)

print('FIN GENERACIÓN DATASET')

instante_final = time.time()

print('Duración TOTAL: ',(instante_final-instante_inicial)/60.,' min')
    
    

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
['environment_workshop.yml', 'udocker_installation', 'data', '.ipynb_checkpoints', 'images', 'overview.ipynb', '.gitignore', 'LECTURAS_segmentacion_cilindrica.py', 'semantic_segmentation_DL.ipynb', 'Deep_Learning', '.git', '__pycache__', 'requirements.ipynb', 'pcd_simulation.ipynb', 'README.md']


<ipython-input-1-c522ce216558>:97: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  m = np.linalg.lstsq(G, z)[0]


['environment_workshop.yml', 'udocker_installation', 'data', '.ipynb_checkpoints', 'images', 'overview.ipynb', '.gitignore', 'LECTURAS_segmentacion_cilindrica.py', 'semantic_segmentation_DL.ipynb', 'Deep_Learning', '.git', '__pycache__', 'requirements.ipynb', 'pcd_simulation.ipynb', 'README.md']
/home/lino/Documentos/PCD_DL_WORKSHOP/PCD_DL_WORKSHOP/data/trees_pcd
Nube generada. Transcurrido por ahora:  0.7997984965642293  min
['environment_workshop.yml', 'udocker_installation', 'data', '.ipynb_checkpoints', 'images', 'overview.ipynb', '.gitignore', 'LECTURAS_segmentacion_cilindrica.py', 'semantic_segmentation_DL.ipynb', 'Deep_Learning', '.git', '__pycache__', 'requirements.ipynb', 'pcd_simulation.ipynb', 'README.md']
/home/lino/Documentos/PCD_DL_WORKSHOP/PCD_DL_WORKSHOP/data/trees_pcd
The folder already exists, so we pass
Nube generada. Transcurrido por ahora:  1.6077455242474874  min
['environment_workshop.yml', 'udocker_installation', 'data', '.ipynb_checkpoints', 'images', 'overview

## 3.2. Training and validation of PointNet++

Now that we have our synthetic dataset created, it's time to run PointNet++ in an **udocker container**. Before starting, some manually work is required. In the file **train_5_layers.py** [located here](./Deep_Learning/pointnet2/scannet/traduccion_lino/debug/Santarem) we will make the following change according to the paths of your specific machine:

<center>
<div class="alert alert-block alert-warning">
    <b>REPLACE:</b> DATASET_PATH should points to your SYNTHETIC_DATASET path 
</div>
</center>

Also, it is required to modify the file named **correr_pointnet_udocker.sh**, which is located in the [PointNet++ directory](./Deep_Learning/pointnet2):

<center>
<div class="alert alert-block alert-warning">
    <b>Line 186:</b> DATASET_PATH should points to your SYNTHETIC_DATASET path 
</div>
</center>


Now we are fully ready to start training and validating the DL model.

Open a terminal and go to the [PointNet++ directory](./Deep_Learning/pointnet2), then run the shell script by doing:

```sh correr_pointnet_udocker.sh```